In [134]:
#import requests
import pandas as pd
#from io import StringIO
#import ipywidgets as widgets
#from IPython.display import display, clear_output
#from datetime import datetime, timedelta
import altair as alt
from vega_datasets import data
#from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, DatePicker
#import geopandas as gpd
#import json
import dash_vega_components as dvc
from dash import Dash, Input, Output, callback, dcc, html
alt.data_transformers.disable_max_rows()
import dash_daq

In [22]:
df  = pd.read_csv('data.csv')

C:\Users\theru\AppData\Local\Temp\ipykernel_29368\2300959333.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv('data.csv')


In [23]:
df.head()

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
0,iscgem16957942,1907-01-02 11:56:08.850000+00:00,"85 km ESE of Neiafu, Tonga",-19.071,-173.306,30.0,7.30,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,820
1,iscgem16957943,1907-01-04 05:19:13.180000+00:00,The 1907 Sumatra Earthquake and Tsunami,2.681,95.530,25.0,8.20,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,1034
2,iscgemsup16957944,1907-01-08 05:21:51.450000+00:00,Scotia Sea,-60.919,-51.954,15.0,6.66,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,682
3,iscgemsup610548676,1907-01-10 05:41:03.810000+00:00,"5 km ESE of Laizhou, China",37.157,119.999,10.0,6.18,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,588
4,iscgemsup610548677,1907-01-12 07:48:30.640000+00:00,"58 km SE of Nikolski, Alaska",52.581,-168.223,15.0,6.35,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,620


In [24]:
df.set_index('id', inplace=True)

In [25]:
df['time'] = pd.to_datetime(df['time'], format = 'ISO8601')

In [26]:
df.isna().sum()

time                  0
location            308
latitude              0
longitude             0
depth               903
magnitude             0
alert           4345404
url                   0
eventtype             0
significance          0
dtype: int64

In [27]:
eq = df.sample(5000, random_state=42)

In [165]:

def create_chart(df, width=800, height=600, 
                 projection ='equalEarth', phi = 0, theta = 0, scale = 100,
                 map_fill = 'darkgrey', map_stroke = 'lightgrey',
                 color_var = 'significance', color_scheme = 'magma',
                 opacity_var = 'magnitude',
                 size_var = 'magnitude', size_range = [10, 200]):
    rotation = [phi, theta, 0]
    Projection = alt.Projection(type = projection, rotate=rotation, scale = scale)

    if color_var == 'time':
        color_var += ':T'
    else:
        color_var += ':Q'
    
    if size_var == 'time':
        size_var += ':T'
    else:
        size_var += ':Q'

    if opacity_var == 'time':
        opacity_var += ':T'
    else:
        opacity_var += ':Q'

    ColorScale = alt.Scale(scheme = color_scheme)
    ColorLegend = alt.Legend(title = color_var)
    Color = alt.Color(color_var, scale = ColorScale, legend=ColorLegend)

    SizeScale = alt.Scale(range=size_range)
    SizeLegend = alt.Legend(title = size_var)
    Size = alt.Size(size_var, scale=SizeScale, legend=SizeLegend)

    OpacityScale = alt.Scale(range = [0.1, 1])
    OpacityLegend = alt.Legend(title = opacity_var)
    Opacity = alt.Opacity(opacity_var, scale=OpacityScale, legend=OpacityLegend)

    topo = alt.topo_feature(data.world_110m.url, 'countries')

    earth = alt.Chart(topo).mark_geoshape(
        fill = map_fill,
        stroke = map_stroke
    ).properties(
        width = width,
        height = height,
        projection = Projection
    )

    graticule = alt.Chart(alt.graticule()).mark_geoshape().properties(projection = Projection)

    earth += graticule

    brush = alt.selection_interval(name = "brush")

    quakes = alt.Chart(df).mark_circle().encode(
        longitude = 'longitude:Q',
        latitude = 'latitude:Q',
        size = Size,
        opacity= Opacity,
        color = alt.condition(brush,
                            Color,
                            alt.value('lightgrey')),
        tooltip = [
            alt.Tooltip('location:N', title='Location'),
            alt.Tooltip('magnitude:Q', title='Magnitude'),
            alt.Tooltip('depth:Q', title='Depth (km)'),
            alt.Tooltip('time:T', title='Time')
        ]
    ).properties(
        projection = Projection
    ).add_params(
        brush
    )   
    return earth + quakes
#create_chart(eq)

In [166]:
from dash_daq import ColorPicker
app = Dash()
app.layout = html.Div([
    html.H1('Title'),
    html.Div(
        ['Projection:',
         dcc.Dropdown(['equalEarth', 'mercator', 'azimuthalEqualArea'], 'mercator', id='proj_dd')]
    ),
    html.Div(
        ['Rotate:',
        dcc.Slider(-179.9,179.9,step=10,value=0,id = 'phi'),
        dcc.Slider(-89.9,89.9,step=10,value=0,id = 'theta')]
    ),
    html.Div(
        ['Scale:', dcc.Slider(100,1000,step=50,value=100,id = 'scale')]
    ),
    html.Div(
        [
            'Map Fill Color:',
            dcc.Input(value = '#4287f5', id='map_fill')
        ]
    ),
    html.Div(
        [
            'Map Stroke Color:',
            dcc.Input(value = 'lightgrey', id='map_stroke')
        ]
    ),
    html.Div(
        [
            'Background Color:',
            dcc.Input(value = '#444444', id='background')
        ]
    ),
    html.Div(
        [
            'Size Variable:',
            dcc.Dropdown(
                options = eq.select_dtypes(include=['number']).columns.tolist(),
                value = 'magnitude',
                id='size_var'
            )
        ]
    ),
    html.Div(
        [
            'Color Variable:',
            dcc.Dropdown(
                options = eq.select_dtypes(include=['number', 'datetime64[ns, UTC]']).columns.tolist(),
                value = 'significance',
                id = 'color_var'
            )
        ]
    ),
     html.Div(
        [
            'Opacity Variable:',
            dcc.Dropdown(
                options = eq.select_dtypes(include=['number', 'datetime64[ns, UTC]']).columns.tolist(),
                value = 'time',
                id = 'opacity_var'
            )
        ]
    ),

    html.Div(id='output_div'),

])

@callback(
    Output('output_div', 'children'),
    Input('proj_dd', 'value'),
    Input('phi', 'value'),
    Input('theta', 'value'),
    Input('scale', 'value'),
    Input('map_fill', 'value'),
    Input('map_stroke', 'value'),
    Input('background', 'value'),
    Input('size_var', 'value'),
    Input('color_var', 'value'),
    Input('opacity_var', 'value')
)
def update_output(proj_dd, phi, theta, scale, map_fill, map_stroke, background, size_var, color_var, opacity_var):
    chart_spec = create_chart(eq, 
                              projection=proj_dd, 
                              phi=phi, theta=theta, 
                              scale = scale, 
                              map_fill=map_fill, 
                              map_stroke = map_stroke,
                              size_var=size_var,
                              color_var=color_var,
                              opacity_var=opacity_var
                              ).properties(
                                  background = background).to_dict()
    return dvc.Vega(
        id='map',
        signalsToObserve=['brush'],
        opt={"renderer": 'svg', 'actions': False},
        spec=chart_spec
    )
app.run()

In [161]:
eq.dtypes

time            datetime64[ns, UTC]
location                     object
latitude                    float64
longitude                   float64
depth                       float64
magnitude                   float64
alert                        object
url                          object
eventtype                    object
significance                  int64
dtype: object